In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Les réseaux à convolutions

Ces réseaux de neurones comprennent deux nouvelles types de couches :
    
- les couches de convolutions
- les couches de pooling

- Bien qu'efficaces pour le traitement d'images, les perceptrons multicouches (MLP) ont des difficultés à gérer des images de grande taille, dû à la croissance exponentielle du nombre de connexions avec la taille de l'image


- Les réseaux de neurones convolutifs, limitent au contraire le nombre de connexions entre un neurone et les neurones des couches adjacentes, ce qui diminue drastiquement le nombre de paramètres à apprendre

http://scs.ryerson.ca/~aharley/vis/conv/flat.html

# Les couches de convolution

La différence fondamentale entre une couche densément connectée et une couche de convolution est la suivante: 

- les couches denses apprennent des motifs globaux dans leur espace de fonctions en entrée (par exemple, pour un chiffre MNIST, des motifs impliquant tous les pixels), tandis que les couches de convolution apprennent des motifs locaux.

Dans le cas des images, des motifs trouvés dans de petites fenêtres 2D des entrées. 

![image](./images/conv.jpg)


Cette caractéristique clé confère à convnets deux propriétés intéressantes:

- Les modèles qu’ils apprennent sont des invariants de traduction.

- Ils peuvent apprendre les hiérarchies spatiales des modèles

Les convolutions sont définies par deux paramètres clés:

- Taille des patchs extraits des entrées

- Profondeur de la carte de caractéristiques en sortie

![image](./images/conv2.jpg)

La couche de convolution est la composante clé des réseaux de neurones convolutifs, et constitue toujours au moins leur première couche.

Son but est de repérer la présence d'un ensemble de features dans les images reçues en entrée. Pour cela, on réalise un filtrage par convolution : le principe est de faire "glisser" une fenêtre représentant la feature sur l'image, et de calculer le produit de convolution entre la feature et chaque portion de l'image balayée. Une feature est alors vue comme un filtre : les deux termes sont équivalents dans ce contexte. 

La couche de convolution reçoit donc en entrée plusieurs images, et calcule la convolution de chacune d'entre elles avec chaque filtre. Les filtres correspondent exactement aux features que l'on souhaite retrouver dans les images. 

On obtient pour chaque paire (image, filtre) une carte d'activation, ou feature map, qui nous indique où se situent les features dans l'image : plus la valeur est élevée, plus l'endroit correspondant dans l'image ressemble à la feature. 

## Comment choisir les features ?

Contrairement aux méthodes traditionnelles, les features ne sont pas pré-définies selon un formalisme particulier, mais apprises par le réseau lors la phase d'entraînement ! Les noyaux des filtres désignent les poids de la couche de convolution. Ils sont initialisés puis mis à jour par rétropropagation du gradient. 

# Les couches de pooling

Chaque couche MaxPooling2D permet de réduire la taille des cartes de caractéristiaues. 

Ce type de couche est souvent placé entre deux couches de convolution : elle reçoit en entrée plusieurs feature maps, et applique à chacune d'entre elles l'opération de pooling. 

L'opération de pooling consiste à réduire la taille des images, tout en préservant leurs caractéristiques importantes.


C’est le rôle du pooling : sous-échantillonner de manière agressive les cartes de features.

# Un cas concret

Nous allons utiliser notre réseau convnet pour classer les chiffres MNIST.

Les 6 lignes de code ci-dessous vous montrent à quoi ressemble un convnet de base. 


C'est une pile de couches `Conv2D` et` MaxPooling2D`


Il est important de noter qu'un convnet prend en entrée des tenseurs de la forme `(image_height, image_width, image_channels)` 


Dans notre cas, nous allons configurer notre convnet pour traiter des entrées de taille `(28, 28, 1)`, qui est le format des images MNIST. 


Nous faisons cela via l'argument `input_shape = (28, 28, 1)`

In [6]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

Voici son architecture :

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Vous pouvez voir que la sortie de chaque couche `Conv2D` et` MaxPooling2D` est un tenseur 3D de forme `(hauteur, largeur, canaux)`. 


Les dimensions en largeur et en hauteur ont tendance à diminuer à mesure que nous progressons dans le réseau. Le nombre de canaux est contrôlé par le premier argument passé aux couches `Conv2D` (par exemple 32 ou 64).


Nous devons donc aplatir nos sorties 3D sur 1D, puis ajouter quelques couches `Dense` au-dessus:

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Maintenant, entraînons notre réseau sur les chiffres du MNIST

In [8]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [9]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
 1920/60000 [..............................] - ETA: 28:28 - loss: 1.3297 - acc: 0.5755

KeyboardInterrupt: 

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [9]:
test_acc

0.99129999999999996

## Le paramétrage des couches

Les couches de convolution et de pooling possèdent des hyperparamètres. 


La taille des feature maps en sortie des couches de convolution et de pooling dépend des hyperparamètres.


Chaque image (ou feature map) est de dimensions W×H×D, où W est sa largeur en pixels, H sa hauteur en pixels et D (1 ou 3).

La couche de convolution possède quatre hyperparamètres :
- Le nombre de filtres K
- La taille F des filtres : chaque filtre est de dimensions F×F×D pixels. 
- Le pas S avec lequel on fait glisser la fenêtre correspondant au filtre sur l'image.
- Le zero-padding  P : on ajoute à l'image en entrée de la couche un contour noir d'épaisseur P pixels. 

La couche de pooling présente seulement deux hyperparamètres :
- La taille F des cellules : l'image est découpée en cellules carrées de taille F×F pixels
- Le pas S : les cellules sont séparées les unes des autres de S pixels




Le choix des hyperparamètres se fait selon un schéma classique :
- Pour la couche de convolution, les filtres sont de petite taille et glissés sur l'image d'un pixel à la fois. On choisit alors F=3,P=1,S=1 ou F=5,P=2,S=1

- Pour la couche de pooling, F=2 et S=2 est un choix judicieux.